In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data['headline']
    date = json_data['datePublished']
    date_m = json_data['dateModified']
    genre = json_data.get('genre', [])
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic not in data:
            data.append(dic)
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [48]:
htmls = get_link(750); htmls

['https://www3.nhk.or.jp/news/html/20190813/k10012032841000.html',
 'https://www3.nhk.or.jp/news/html/20190813/k10012032991000.html',
 'https://www3.nhk.or.jp/news/html/20190813/k10012033071000.html',
 'https://www3.nhk.or.jp/news/html/20190813/k10012033431000.html',
 'https://www3.nhk.or.jp/news/html/20190813/k10012033471000.html',
 'https://www3.nhk.or.jp/news/html/20190813/k10012034021000.html',
 'https://www3.nhk.or.jp/news/html/20190814/k10012034201000.html',
 'https://www3.nhk.or.jp/news/html/20190814/k10012035261000.html',
 'https://www3.nhk.or.jp/news/html/20190814/k10012035421000.html',
 'https://www3.nhk.or.jp/news/html/20190815/k10012035821000.html',
 'https://www3.nhk.or.jp/news/html/20190815/k10012035841000.html',
 'https://www3.nhk.or.jp/news/html/20190815/k10012036341000.html',
 'https://www3.nhk.or.jp/news/html/20190815/k10012036721000.html',
 'https://www3.nhk.or.jp/news/html/20190815/k10012036871000.html',
 'https://www3.nhk.or.jp/news/html/20190815/k10012037231000.ht

# scrape

In [20]:
driver = webdriver.Chrome()

In [49]:
for nhkurl in tqdm.tqdm(htmls[:-2]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)







  0%|          | 0/20 [00:00<?, ?it/s]





  5%|▌         | 1/20 [01:00<19:06, 60.34s/it]





 10%|█         | 2/20 [01:56<17:44, 59.17s/it]





 15%|█▌        | 3/20 [02:56<16:49, 59.36s/it]





 20%|██        | 4/20 [03:57<15:56, 59.76s/it]





 25%|██▌       | 5/20 [04:53<14:38, 58.56s/it]





 30%|███       | 6/20 [05:52<13:44, 58.86s/it]





 35%|███▌      | 7/20 [06:52<12:50, 59.27s/it]





 40%|████      | 8/20 [07:58<12:13, 61.14s/it]





 45%|████▌     | 9/20 [08:58<11:08, 60.78s/it]





 50%|█████     | 10/20 [10:01<10:15, 61.57s/it]





 55%|█████▌    | 11/20 [11:02<09:12, 61.43s/it]





 60%|██████    | 12/20 [12:04<08:13, 61.63s/it]





 65%|██████▌   | 13/20 [13:13<07:26, 63.81s/it]





 70%|███████   | 14/20 [14:27<06:40, 66.83s/it]





 75%|███████▌  | 15/20 [15:33<05:33, 66.67s/it]





 80%|████████  | 16/20 [16:46<04:34, 68.51s/it]





 85%|████████▌ | 17/20 [17:41<03:13, 64.40s/it]





 90%|█████████ | 18/20 [18:41<02:05, 62.92s/it]





 95%|

In [44]:
html = driver.page_source.encode('utf-8')
dic = scrape(html)
js(dic)

In [15]:
re.findall(r'1 time(.............................)', html)

['</strong>&nbsp;<a href="/web/']